In [79]:
suppressMessages(require(Rtsne))
suppressMessages(require(cellrangerRkit))
suppressMessages(require(ggplot2))
suppressMessages(require(data.table))
suppressMessages(require(gplots))
suppressMessages(require(matrixStats))
suppressMessages(require(RColorBrewer))
#suppressMessages(require(RUVnormalize))
#suppressMessages(require(SCnorm))
require(mixtools)

pMannWhitne=function(group1,group2){
    p=sapply(seq(ncol(group1)),function(x){wilcox.test(group1[,x], group2[,x],exact=F)$p.value})
    foldchange=sapply(seq(ncol(group1)),function(x){log2(mean(group1[,x]))-log2(mean(group2[,x]))})
    o=cbind(p,foldchange)
    colnames(o)=c("pval","foldchange")
    return(p)
}

Mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

substrRight <- function(x, n){
  substr(x, nchar(x)-n+1, nchar(x))
}

upfirstletter <- function(s){
    paste0(toupper(substring(s, 1,1)), tolower(substring(s, 2)))
}

duplicate_alterning=function(df,altersd=0.1){
    tagduplicated=which(duplicated(df))
    if(length(tagduplicated)>0){
        df[tagduplicated,]=df[tagduplicated,]+rnorm(ncol(df)*length(tagduplicated), 0, altersd)
    }
    return(df)
}

idconversion=data.frame(old=c("2B4-CD244","C-KIT-CD117","CD103","CD11A","CD11B","CD11C","CD122","CD123","CD127","CD16","CD161","CD25","CD29","CD34","CD39","CD45","CD45RA","CD45RO","CD49A","CD49D","CD56","CD57","CD62L","CD94","CD95","CRTH2-CD294","CTLA4-CD152","CXCR3-CD183","HLA-DR","IL21R-CD360","NKP46-CD335","OX40","PDL-1-CD274","TCRAB","TCRGD","TIM-3","VA7-2","VD2","B7","GP130"),
                        new=c("CD244","KIT","ITGAE","ITGAL","ITGAM","ITGAX","IL2RB","IL3RA","IL7R","FCGR3A","KLRB1","IL2RA","ITGB1","CD34","ENTPD1","PTPRC","PTPRC","PTPRC","ITGA1","ITGA4","NCAM1","B3GAT1","SELL","KLRD1","FAS","PTGDR2","CTLA4","CXCR3","HLA-DRA","IL21R","NCR1","TNFRSF4","PDL1","TRAC","TRDC","HAVCR2","TRAV7","VD2","CD80","IL6ST"),
                        stringsAsFactors=F)

#parameter setting
genome <- "GRCh38"
min_genenum=20 #minimun gene number required per cell, used in filtering the ball
cell_per_sample=200
nKeep=5000
cellmax=10000
cellrangerOutDir="/home/ahe/Analysis/201801_JohnVDJ/data/10x"
proteinDir="/home/ahe/Analysis/201801_JohnVDJ/data/protein"

proteinfiles=list.files(path = proteinDir, pattern = "_AOC.matrix.umi.txt", full.names = T)
samplesnames=gsub(paste0(proteinDir,"/"),"",gsub("_AOC.matrix.umi.txt","",proteinfiles))
tagdir=paste0(cellrangerOutDir,"/",samplesnames,"_10x")

In [87]:
#combining of all single library
set.seed(123)
gene_bc_matrix=list()
protein_mat=list()
for(i in 1:length(tagdir)){
    #readin cells
    gene_bc_matrix[[i]]=load_cellranger_matrix_h5(tagdir[i], genome=genome,barcode_filtered =F)
    gene_bc_matrix[[i]]=exprs(gene_bc_matrix[[i]])
    gene_bc_matrix[[i]]=as.matrix(gene_bc_matrix[[i]][,colSums(gene_bc_matrix[[i]])>50]) #the filtering need to be tinkered with
    gene_bc_matrix[[i]]=gene_bc_matrix[[i]][,order(colSums(gene_bc_matrix[[i]]),decreasing = T)]
    #filter out the "ball"
    temp=t(gene_bc_matrix[[i]][rowSums(gene_bc_matrix[[i]])>ncol(gene_bc_matrix[[i]])/100,]) # store cells with filtered gene num
    genenum=rowSums(temp>2)
    gene_bc_matrix[[i]]=gene_bc_matrix[[i]][,genenum>=50] 
    gene_bc_matrix[[i]]=t(gene_bc_matrix[[i]])
    
    #read protein data
    protein_mat[[i]]=fread(proteinfiles[i],data.table=F)
    rownames(protein_mat[[i]])=protein_mat[[i]][,1]
    protein_mat[[i]]=protein_mat[[i]][,-1]
    rownames(gene_bc_matrix[[i]])=gsub("1$", i, rownames(gene_bc_matrix[[i]]))
    rownames(protein_mat[[i]])=gsub("1$", i, rownames(protein_mat[[i]]))
    
    #some other statistics
    cat(paste0(i,"\n"))
}
ExpressionMat=do.call(rbind,gene_bc_matrix)
ProteinMat=do.call(rbind,protein_mat)
rightorder=1:length(tagdir)
rm(gene_bc_matrix)

#filter the unused gene
ExpressionMat=ExpressionMat[,colSums(ExpressionMat)>2]

#filter based on UMI
#only the gene that have 1 UMI in more than 1% of cells are counted as valid
ExpressionBinaryMat=ExpressionMat>=1
ExpressionMat=ExpressionMat[,colSums(ExpressionBinaryMat)>nrow(ExpressionMat)/100]
rm(ExpressionBinaryMat)

#remove duplicate cells
ExpressionMat=duplicate_alterning(ExpressionMat)
ProteinMat=duplicate_alterning(ProteinMat)

#change gene name
#load gene names to the object

mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
  dataset = "hsapiens_gene_ensembl",  #human
  host = 'ensembl.org')
#biomaRt::listAttributes(mart)
t2g <- biomaRt::getBM(attributes = c("ensembl_transcript_id", "ensembl_gene_id",
    "external_gene_name","chromosome_name","start_position","end_position","strand"), mart = mart)
t2g <- dplyr::rename(t2g, target_id = ensembl_transcript_id,
  ens_gene = ensembl_gene_id, ext_gene = external_gene_name)

#change name
ExpressionMat=ExpressionMat[,which(colnames(ExpressionMat) %in% t2g$ens_gene)]
colnames(ExpressionMat)=t2g$ext_gene[match(colnames(ExpressionMat),t2g$ens_gene)]
colnames(ExpressionMat)[which(duplicated(colnames(ExpressionMat)))]=paste0(colnames(ExpressionMat)[which(duplicated(colnames(ExpressionMat)))],"_1")

#change name for protein
colnames(ProteinMat)=toupper(colnames(ProteinMat))
colnames(ProteinMat)[colnames(ProteinMat) %in% idconversion$old]=idconversion$new[match(colnames(ProteinMat)[colnames(ProteinMat) %in% idconversion$old],idconversion$old)]

1
2
3
4
5
6


In [83]:
ProteinMat_new=ProteinMat[rownames(ProteinMat) %in% rownames(ExpressionMat),]
ExpressionMatt_new=ExpressionMat[rownames(ExpressionMat) %in% rownames(ProteinMat),]
ProteinMat_new=ProteinMat_new[match(rownames(ExpressionMatt_new),rownames(ProteinMat_new)),]
batch=as.numeric(substrRight(rownames(ExpressionMatt_new),1))

#z score by column(gene)
ExpressionMat_norm=t(apply(ExpressionMatt_new,1,function(x){x/sum(x)*1000000}))
ExpressionMat_norm=scale(ExpressionMat_norm)

ProteinMat_norm=t(apply(ProteinMat_new,1,function(x){x/sum(x)*1000000}))
ProteinMat_norm=scale(ProteinMat_norm)
ProteinMat_norm[,is.na(colSums(ProteinMat_norm))]=0

In [6]:
colnames(ProteinMat)[colnames(ProteinMat) %in% colnames(ExpressionMat)]

[1] "CCR1"   "CCR2"   "CCR4"   "CCR6"   "CCR9"   "CD19"   "CD27"   "CD28"  
 [9] "CD38"   "CD3E"   "CD4"    "CD44"   "CD69"   "CD7"    "CD8A"   "CX3CR1"
[17] "CXCR5"  "ICOS"   "IL15RA" "IL23R"  "KLRG1"  "TIGIT"  "CCR7"

In [88]:
sum(rownames(ProteinMat) %in% rownames(ExpressionMat))

[1] 27

In [91]:
nrow(ProteinMat)

[1] 241

In [98]:
Colpallete=colorRampPalette(brewer.pal(min(9,length(tagdir)), "Set1"))
coltouse=Colpallete(max(batch))

pdf("protein_rna_correlation.pdf",width=12,height=12)
par(mfrow=c(4,4))
for(i in 1:ncol(ProteinMat_new)){
    tagprotein=ProteinMat_new[,i]
    if(length(which(colnames(ExpressionMatt_new)==colnames(ProteinMat_new)[i]))){
        tagrna=ExpressionMatt_new[,which(colnames(ExpressionMatt_new)==colnames(ProteinMat_new)[i])]
    }else{
        tagrna=rep(0,nrow(ProteinMat_new))
    }
    plot(tagprotein,tagrna,main=paste0("raw:",colnames(ProteinMat_norm)[i]),pch=19,cex=0.5,xlab="protein level",ylab="RNA level",col=coltouse[batch])
}

dev.off()

pdf("protein_RNA_totalreads_per_cell.pdf",width=12,height=6)
par(mfrow=c(1,2))
plot(rowSums(ProteinMat_new),rowSums(ExpressionMatt_new),pch=19,cex=0.5,xlim=c(0,200),col=coltouse[batch],xlab="protein reads/cell",ylab="RNA reads/cell",main="raw")
dev.off()

png 
  2

png 
  2

In [95]:
tagrna

CTGGTCTAGCCACGCT-1 CGTCTACTCACAACGT-1 CTTACCGTCCGAGCCA-1 CCACTACCACCGGAAA-2 
                 1                  0                  0                  0 
CGTGTCTAGTTGAGAT-2 GAACGGACACATGACT-2 TACTCGCTCGGTGTTA-2 CTCATTAGTGCGAAAC-2 
                 0                  1                  0                  0 
GACGTTAAGGACAGAA-2 TCAATCTTCCATGAGT-3 CTGGTCTTCGTTGACA-4 CATCGGGCACCGTTGG-4 
                 0                  1                  2                  0 
TGTGTTTAGCCCAGCT-4 GCGCAACGTGGAAAGA-4 TCGGTAACACGGTAGA-4 TATCAGGCATAGGATA-4 
                 0                  0                  1                  0 
GGGTTGCCATTACCTT-4 GGATTACCATAACCTG-4 TTCGAAGCATGAAGTA-4 TTCTTAGTCATGTCTT-4 
                 1                  0                  0                  1 
TCAATCTTCAACTCTT-4 AAAGCAAGTTTACTCT-4 GGGCACTGTGAACCTT-4 GGCAATTCATGTAGTC-4 
                 1                  0                  0                  0 
TACTCATCACATCCGG-4 ACTGAGTCAGGAATGC-6 GCTGGGTAGTGACATA-6 
                 0                  0                  2